Import Classes

In [1]:
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from keras.datasets import cifar100
import tensorflow as tf
from keras.utils import np_utils
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


Eraser

In [0]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

Prepare the dataset

In [3]:
num_classes = 100
nb_epochs = 10

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

raw_x_test = x_test
raw_y_test = y_test

#Pre-process the data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))
datagen.fit(x_train)

169009152/169001437 [==============================] - 4s 0us/step


In [0]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

Form the model

In [5]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

94773248/94765736 [==============================] - 1s 0us/step


Training

In [7]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
batch_size = 64
t=time.time()
historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                  batch_size=batch_size),
                                  steps_per_epoch=x_train.shape[0] // batch_size,
                                  epochs=20,
                                  validation_data=(x_test, y_test))
print('Training time: %s' % (t - time.time()))

Epoch 1/20
781/781 [==============================] - 262s 336ms/step - loss: 1.8463 - accuracy: 0.5153 - val_loss: 1.0000 - val_accuracy: 0.6996
Epoch 2/20
781/781 [==============================] - 261s 334ms/step - loss: 1.1386 - accuracy: 0.6722 - val_loss: 0.8123 - val_accuracy: 0.7559
Epoch 3/20
781/781 [==============================] - 260s 334ms/step - loss: 0.9551 - accuracy: 0.7202 - val_loss: 0.7820 - val_accuracy: 0.7640
Epoch 4/20
781/781 [==============================] - 261s 334ms/step - loss: 0.8646 - accuracy: 0.7411 - val_loss: 0.7321 - val_accuracy: 0.7757
Epoch 5/20
781/781 [==============================] - 261s 334ms/step - loss: 0.7931 - accuracy: 0.7619 - val_loss: 0.7408 - val_accuracy: 0.7844
Epoch 6/20
781/781 [==============================] - 261s 334ms/step - loss: 0.7364 - accuracy: 0.7774 - val_loss: 0.7182 - val_accuracy: 0.7860
Epoch 7/20
781/781 [==============================] - 261s 334ms/step - loss: 0.6889 - accuracy: 0.7896 - val_loss: 0.7066 -

Prediction & Evaluation

In [0]:
pred = model.predict(x = x_test, batch_size = 64)

In [9]:
from PIL import Image
import matplotlib.pyplot as plt


CIFAR100_LABELS_LIST = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]


from sklearn.metrics import confusion_matrix

y_pred = []
y_true = []
trues = 0

length = len(pred)
preds = np.argmax(pred, axis =1)
for j in range(length):
    prediction = CIFAR100_LABELS_LIST[preds[j]]
    true_label = CIFAR100_LABELS_LIST[raw_y_test[j][0]]
    y_pred.append(prediction)
    y_true.append(true_label)
    if(prediction == true_label):
        trues += 1

print("Final accuracy:", trues / len(preds))

cm = confusion_matrix(y_true, y_pred, labels=CIFAR100_LABELS_LIST)
print(cm)


Final accuracy: 0.8015
[[98  0  0 ...  0  0  0]
 [ 0 94  0 ...  0  0  0]
 [ 0  0 64 ...  0  2  0]
 ...
 [ 0  0  0 ... 82  0  0]
 [ 0  0  1 ...  1 63  0]
 [ 0  0  0 ...  0  0 86]]
